REMEMBER: FIRST CREATE A COPY OF THIS FILE WITH A UNIQUE NAME AND DO YOUR WORK THERE. AND MAKE SURE YOU COMMIT YOUR CHANGES TO THE `hw3_submissions` BRANCH.

# Assignment 3 | Cleaning and Exploring Data with Pandas



<img src="data/scoreCard.jpg" width=250>

In this assignment, you will investigate restaurant food safety scores for restaurants in San Francisco. Above is a sample score card for a restaurant. The scores and violation information have been made available by the San Francisco Department of Public Health. 

## Loading Food Safety Data


There are 2 files in the data directory:
1. business.csv containing food establishments in San Francisco
1. inspections.csv containing retaurant inspections records

Let's start by loading them into Pandas dataframes.  One of the files, business.csv, has encoding (ISO-8859-1), so you will need to account for that when reading it.

### Question 1

#### Question 1a
Read the two files noted above into two pandas dataframes named `bus` and `ins`, respectively. Print the first 5 rows of each to inspect them.


In [1]:
import pandas as pd

In [2]:
bus = pd.read_csv('data/businesses.csv', encoding='ISO-8859-1')
ins = pd.read_csv('data/inspections.csv')

In [3]:
bus.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,phone_number
0,10,TIRAMISU KITCHEN,033 BELDEN PL,San Francisco,CA,94104,37.791116,-122.403816,+14154217044
1,19,NRGIZE LIFESTYLE CAFE,"1200 VAN NESS AVE, 3RD FLOOR",San Francisco,CA,94109,37.786848,-122.421547,+14157763262
2,24,OMNI S.F. HOTEL - 2ND FLOOR PANTRY,"500 CALIFORNIA ST, 2ND FLOOR",San Francisco,CA,94104,37.792888,-122.403135,+14156779494
3,31,NORMAN'S ICE CREAM AND FREEZES,2801 LEAVENWORTH ST,San Francisco,CA,94133,37.807155,-122.419004,NaN
4,45,CHARLIE'S DELI CAFE,3202 FOLSOM ST,San Francisco,CA,94110,37.747114,-122.413641,+14156415051


In [4]:
ins.head()

,business_id,score,date,type
0,10,82,20160503,routine
1,10,94,20140729,routine
2,10,92,20140114,routine
3,19,94,20160513,routine
4,19,94,20141110,routine


## Examining the Business data

From its name alone, we expect the `businesses.csv` file to contain information about the restaurants. Let's investigate this dataset.

### Question 2

#### Question 2a: How many records are there?

In [5]:
len(bus)

6315

#### Question 2b: How many unique business IDs are there?  

In [6]:
len(bus['business_id'].value_counts())

6315

**R/:** Every record in the business data is unique

#### Question 2c: What are the 5 most common businesses by name, and how many are there in San Francisco?

In [7]:
bus[['business_id', 'name']].groupby('name').count().sort_values('business_id', ascending = False).head(5)

,business_id
name,
STARBUCKS COFFEE,72
PEET'S COFFEE & TEA,24
MCDONALDS,12
WALGREENS,11
SAN FRANCISCO SOUP COMPANY,11


## Zip code

Next, let's  explore some of the variables in the business table. We begin by examining the postal code.

### Question 3

#### Question 3a
How are the zip code values stored in python (i.e. data type)?

To answer this you might want to examine a particular entry.

In [8]:
bus.dtypes

business_id       int64
name             object
address          object
city             object
state            object
postal_code      object
latitude        float64
longitude       float64
phone_number     object
dtype: object

**R/:** The default data type for zip code in the businesses dataframe is "object", that corresponds to a string or mix type of data. In the answer to the following question, you can see that there are numbers and characters in the column, both stored as strings or a “mix”.

#### Question 3b

What are the unique values of postal_code?

In [9]:
bus['postal_code'].value_counts()

94110        585
94103        558
94102        458
94107        451
94133        417
94109        383
94111        282
94122        270
94118        243
94115        238
94114        230
94105        226
94108        224
94117        197
94112        190
94123        169
94124        164
94121        161
94132        134
94104        133
94116         93
94134         74
94127         71
94131         48
94158         17
94130          8
94143          5
CA             3
Ca             3
94013          2
94129          2
94014          2
94101          2
95105          1
94188          1
941102019      1
94609          1
94602          1
94120          1
00000          1
92672          1
94621          1
94545          1
94066          1
941033148      1
941            1
Name: postal_code, dtype: int64

#### Question 3c

Let's say we decide to exclude the businesses that have no zipcode for our analysis (which might include food trucks for example).  Use the list of valid 5-digit zip codes below to create a new dataframe called bus_valid, with only businesses whose postal_codes show up in this list of valid zipcodes. How many businesses are there in this new dataframe?

In [10]:
validZip = ["94102", "94103", "94104", "94105", "94107", "94108",
            "94109", "94110", "94111", "94112", "94114", "94115",
            "94116", "94117", "94118", "94121", "94122", "94123", 
            "94124", "94127", "94131", "94132", "94133", "94134"]

In [11]:
bus2 = bus[bus['postal_code'].isin(validZip)]

In [12]:
# Number of businesses in the new dataframe after exluding those with no valid 5-digit zip code
len(bus2['business_id'].value_counts())    

5999

In [13]:
# number of businesses in the new dataframe by zip code
bus2['postal_code'].value_counts()

94110    585
94103    558
94102    458
94107    451
94133    417
94109    383
94111    282
94122    270
94118    243
94115    238
94114    230
94105    226
94108    224
94117    197
94112    190
94123    169
94124    164
94121    161
94132    134
94104    133
94116     93
94134     74
94127     71
94131     48
Name: postal_code, dtype: int64

## Latitude and Longitude

Another aspect of the data we want to consider is the prevalence of missing values. If many records have missing values then we might be concerned about whether the nonmissing values are representative of the population.

### Question 4
 
Consider the longitude and latitude in the business DataFrame. 

#### Question 4a

How many businesses are missing longitude values, working with only the businesses that are in the list of valid zipcodes?

In [14]:
bus2[pd.isnull(bus2['longitude'])]

,business_id,name,address,city,state,postal_code,latitude,longitude,phone_number
200,821,LA SANTANECA,3781 MISSION ST,San Francisco,CA,94110,NaN,NaN,+14156481034
210,852,NEW HOLLYWOOD BAKERY & RESTAURANT,652 PACIFIC AVE,San Francisco,CA,94133,NaN,NaN,+14153979919
760,2934,LOU'S FISH SHACK,300 JEFFERSON ST,San Francisco,CA,94133,NaN,NaN,+14157715687
826,3123,LES JOULINS,44 ELLIS ST,San Francisco,CA,94102,NaN,NaN,+14153975397
1100,4323,DIVA'S,1081 POST ST,San Francisco,CA,94109,NaN,NaN,+14154743482
...,...,...,...,...,...,...,...,...,...
6310,90234,ACCESS SFUSD THE ARC,1520 HOWARD ST,San Francisco,CA,94103,NaN,NaN,NaN
6311,90236,PHO DONG HUONG,667 MONTEREY BLVD,San Francisco,CA,94127,NaN,NaN,+14154945048
6312,90237,WIN'S RESTAURANT,3040 TARAVAL ST,San Francisco,CA,94116,NaN,NaN,+14158606036
6313,90238,USA POWER MARKET,1524 OCEAN AVE,San Francisco,CA,94112,NaN,NaN,+14153331878


In [15]:
len(bus2[pd.isnull(bus2['longitude'])])

2483

#### Question 4b

Create a new dataframe with one row for each valid zipcode. The dataframe should include the following three columns:

1. `postal_code`: Contains the zip codes in the `validZip` variable above.
2. `null_lon`: The number of businesses in that zipcode with missing `longitude` values.
3. `not_null_lon`: The number of businesses without missing `longitude` values.

In [16]:
# count of total business id's by zip code
bus_zc = bus2[['business_id','postal_code']].rename(
    columns={'business_id': 'total_bus'}).groupby('postal_code', as_index = False).count()
bus_zc

,postal_code,total_bus
0,94102,458
1,94103,558
2,94104,133
3,94105,226
4,94107,451
5,94108,224
6,94109,383
7,94110,585
8,94111,282
9,94112,190


In [18]:
# count of business id's by zipcode with no null values of longitude
bus_zc_temp = bus2[['longitude','postal_code']].rename(
    columns = {'longitude': 'not_null_lon'}).groupby('postal_code', dropna = True, as_index = False).count()
bus_zc_temp

,postal_code,not_null_lon
0,94102,262
1,94103,306
2,94104,74
3,94105,126
4,94107,204
5,94108,142
6,94109,237
7,94110,334
8,94111,167
9,94112,131


In [19]:
# merge both dataframes
bus_zc2 = pd.merge(bus_zc, bus_zc_temp, on = "postal_code")
bus_zc2

,postal_code,total_bus,not_null_lon
0,94102,458,262
1,94103,558,306
2,94104,133,74
3,94105,226,126
4,94107,451,204
5,94108,224,142
6,94109,383,237
7,94110,585,334
8,94111,282,167
9,94112,190,131


In [20]:
# add new column to substract and obtain number of businesses with null longitude values
bus_zc2['null_lon'] = bus_zc2["total_bus"] - bus_zc2["not_null_lon"]
bus_zc2

,postal_code,total_bus,not_null_lon,null_lon
0,94102,458,262,196
1,94103,558,306,252
2,94104,133,74,59
3,94105,226,126,100
4,94107,451,204,247
5,94108,224,142,82
6,94109,383,237,146
7,94110,585,334,251
8,94111,282,167,115
9,94112,190,131,59


In [21]:
# delete column of the total number of businesses
bus_zc3 = bus_zc2.drop(['total_bus'], axis = 1)
bus_zc3

,postal_code,not_null_lon,null_lon
0,94102,262,196
1,94103,306,252
2,94104,74,59
3,94105,126,100
4,94107,204,247
5,94108,142,82
6,94109,237,146
7,94110,334,251
8,94111,167,115
9,94112,131,59


#### 4c. Do any zip codes appear to have more than their 'fair share' of missing longitude? 

To answer this, you will want to compute the proportion of missing longitude values for each zip code, and print the proportion missing longitude, and print the top five zipcodes in descending order of proportion missing postal_code.


In [22]:
bus_zc3['prop_null_lon'] = bus_zc3['null_lon'] / (bus_zc3['null_lon'] + bus_zc3['not_null_lon'])
bus_zc3.sort_values('prop_null_lon', ascending = False).head(5)

,postal_code,not_null_lon,null_lon,prop_null_lon
4,94107,204,247,0.547672
18,94124,76,88,0.536585
21,94132,69,65,0.485075
10,94114,122,108,0.469565
16,94122,147,123,0.455556


# Investigate the inspection data

Let's now turn to the inspection DataFrame. Earlier, we found that `ins` has 4 columns, these are named `business_id`, `score`, `date` and `type`.  In this section, we determine the granularity of `ins` and investigate the kinds of information provided for the inspections. 

### Question 5

#### Question 5a
As with the business data, assess whether there is one inspection record for each business, by counting how many rows are in the data and how many unique businesses there are in the data. If they are exactly the same number, it means there is only one inspection per business, clearly.

In [23]:
ins.head()

,business_id,score,date,type
0,10,82,20160503,routine
1,10,94,20140729,routine
2,10,92,20140114,routine
3,19,94,20160513,routine
4,19,94,20141110,routine


In [24]:
len(ins)

15430

In [25]:
len(ins['business_id'].value_counts())

5730

**R/:** The number of records is not the same as the number of unique businesses id's, meaning that there can be more than one inspection record per business.

#### Question 5b

What values does `type` take on? How many occurrences of each value is in the DataFrame? Create a new dataframe named `ins2` by copying `ins` and keeping only records with values of `type` that occur more than 10 times in the original table. In other words, eliminate records that have values of `type` that occur rarely (< 10 times). Check the result to make sure rare types are eliminated.

In [26]:
ins['type'].value_counts()

routine      15429
complaint        1
Name: type, dtype: int64

**R/:** type only takes two values in the inspections dataframe: "routine" or "compliant"

In [27]:
# group inspections data by type
ins_type = ins[['business_id','type']].rename(
    columns={'business_id': 'total_ins'}).groupby('type', as_index = False).count()

# inspection types with at least 10 records
ins_type = ins_type['type'][ins_type['total_ins'] >= 10]
ins_type

1    routine
Name: type, dtype: object

In [28]:
# drop records with rare types
ins2 = ins[ins['type'].isin(ins_type)]
ins2

,business_id,score,date,type
0,10,82,20160503,routine
1,10,94,20140729,routine
2,10,92,20140114,routine
3,19,94,20160513,routine
4,19,94,20141110,routine
...,...,...,...,...
15425,89515,98,20161103,routine
15426,89701,100,20161206,routine
15427,89790,96,20161129,routine
15428,89803,100,20161215,routine


#### Question 5c

Since the data was stored in a .csv file, the dates are formatted as strings such as `20160503`. Once we read in the data, we would like to have dates in an appropriate format for analysis. Add a new column called `year` by capturing the first four characters of the date column. 

Hint: we have seen multiple ways of doing this in class, includings `str` operations, `lambda` functions, `datetime` operations, and others. Choose the method that works best for you :)

In [29]:
ins2.dtypes

business_id     int64
score           int64
date            int64
type           object
dtype: object

In [30]:
ins2['year'] = pd.to_datetime(ins2['date'], format='%Y%m%d')
ins2['year'] = pd.DatetimeIndex(ins2['year']).year
ins2

<ipython-input-30-c552d4c5736d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ins2['year'] = pd.to_datetime(ins2['date'], format='%Y%m%d')
<ipython-input-30-c552d4c5736d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ins2['year'] = pd.DatetimeIndex(ins2['year']).year


,business_id,score,date,type,year
0,10,82,20160503,routine,2016
1,10,94,20140729,routine,2014
2,10,92,20140114,routine,2014
3,19,94,20160513,routine,2016
4,19,94,20141110,routine,2014
...,...,...,...,...,...
15425,89515,98,20161103,routine,2016
15426,89701,100,20161206,routine,2016
15427,89790,96,20161129,routine,2016
15428,89803,100,20161215,routine,2016


#### Question 5d

What range of years is covered in this data set? Are there roughly same number of inspections each year? Try dropping records for any years with less than 50 inspections and store the result in a new dataframe named `ins3`.

In [31]:
ins_year = ins2[['business_id', 'year']].groupby('year', as_index = False).count().rename(
    columns = {'business_id': 'inspections'})
ins_year

,year,inspections
0,2013,38
1,2014,5629
2,2015,3923
3,2016,5839


In [32]:
ins_year = ins_year['year'][ins_year['inspections'] >= 50]
ins_year

1    2014
2    2015
3    2016
Name: year, dtype: int64

In [33]:
# drop records from years with less than 50 inspections
ins3 = ins2[ins2['year'].isin(ins_year)]
ins3

,business_id,score,date,type,year
0,10,82,20160503,routine,2016
1,10,94,20140729,routine,2014
2,10,92,20140114,routine,2014
3,19,94,20160513,routine,2016
4,19,94,20141110,routine,2014
...,...,...,...,...,...
15425,89515,98,20161103,routine,2016
15426,89701,100,20161206,routine,2016
15427,89790,96,20161129,routine,2016
15428,89803,100,20161215,routine,2016


Let's examine only the inspections for one year: 2016. This puts businesses on a more equal footing because [inspection guidelines](https://www.sfdph.org/dph/eh/Food/Inspections.asp) generally refer to how many inspections should occur in a given year.

In [34]:
ins2016 = ins3[ins3['year'] == 2016]
ins2016

,business_id,score,date,type,year
0,10,82,20160503,routine,2016
3,19,94,20160513,routine,2016
6,24,98,20161005,routine,2016
7,24,96,20160311,routine,2016
10,31,84,20160816,routine,2016
...,...,...,...,...,...
15425,89515,98,20161103,routine,2016
15426,89701,100,20161206,routine,2016
15427,89790,96,20161129,routine,2016
15428,89803,100,20161215,routine,2016


### Question 6

#### Question 6a

Merge the business and 2016 inspections data, keeping all businesses regardless of whether they show up in the inspections file. Show the first several rows of the resulting dataframe.

In [35]:
bus3 = pd.merge(bus2, ins2016, on = "business_id")
bus3

,business_id,name,address,city,state,postal_code,latitude,longitude,phone_number,score,date,type,year
0,10,TIRAMISU KITCHEN,033 BELDEN PL,San Francisco,CA,94104,37.791116,-122.403816,+14154217044,82,20160503,routine,2016
1,19,NRGIZE LIFESTYLE CAFE,"1200 VAN NESS AVE, 3RD FLOOR",San Francisco,CA,94109,37.786848,-122.421547,+14157763262,94,20160513,routine,2016
2,24,OMNI S.F. HOTEL - 2ND FLOOR PANTRY,"500 CALIFORNIA ST, 2ND FLOOR",San Francisco,CA,94104,37.792888,-122.403135,+14156779494,98,20161005,routine,2016
3,24,OMNI S.F. HOTEL - 2ND FLOOR PANTRY,"500 CALIFORNIA ST, 2ND FLOOR",San Francisco,CA,94104,37.792888,-122.403135,+14156779494,96,20160311,routine,2016
4,31,NORMAN'S ICE CREAM AND FREEZES,2801 LEAVENWORTH ST,San Francisco,CA,94133,37.807155,-122.419004,NaN,84,20160816,routine,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5628,89475,LOVEJOY'S TEA ROOM,1351 CHURCH ST,San Francisco,CA,94114,NaN,NaN,+14156486845,96,20161130,routine,2016
5629,89495,THE SARAP SHOP LLC,428 11TH ST,San Francisco,CA,94103,NaN,NaN,+14157430747,100,20161008,routine,2016
5630,89515,ONE WAAN THAI,2922 DIAMOND ST,San Francisco,CA,94131,NaN,NaN,+14157136773,98,20161103,routine,2016
5631,89701,THOMAS EDISON CHARTER ACADEMY,3531 22 ST,San Francisco,CA,94114,NaN,NaN,+14159703330,100,20161206,routine,2016


#### Question 6b
Print the 20 lowest rated businesses names, their addresses, and their ratings.

In [36]:
bus4 = bus3[["name", "address", "score"]]
bus4.sort_values('score').head(20)

,name,address,score
3842,GOLDEN RIVER RESTAURANT,5827 GEARY BLVD,52
3828,UNCLE CAFE,65 WAVERLY PL,55
4833,CRAZY PEPPER,2257 SAN JOSE AVE,55
5536,POKI TIME,2101 LOMBARD ST,55
4368,GOLDEN WOK,295 B ORIZABA AVE,56
2463,HING WANG BAKERY,339 JUDAH ST,56
696,RED A BAKERY,634 CLEMENT ST,57
4694,L & G VIETNAMESE SANDWICH,602 EDDY ST,57
3350,SAN TUNG RESTAURANT LLC,1031 IRVING ST,57
3851,"NEW GARDEN RESTAURANT, INC.",716 KEARNY ST,57


## Done!

Now commit this notebook to your `hw3_submissions` branch, push it to your GitHub repo, and open a PR!